<a href="https://colab.research.google.com/github/radicalcoder/ML-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd


# Allow image embeding in notebook
%matplotlib inline
!git clone -l -s https://github.com/radicalcoder/ML-Project.git data
%cd data
!ls

Cloning into 'data'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 7370 (delta 11), reused 0 (delta 0), pack-reused 7349
Receiving objects: 100% (7370/7370), 247.13 MiB | 32.22 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Checking out files: 100% (7225/7225), done.
/content/data
BelgiumTSC_Testing   MLP_final_initialdraft.ipynb  README.md
BelgiumTSC_Training  MLP.ipynb			   traffic.py


In [7]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    print(directories)
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels


# Load training and testing datasets.
train_data_dir = "BelgiumTSC_Training/Training/"
test_data_dir = "BelgiumTSC_Testing/Testing/"

images, labels = load_data(train_data_dir)

['00055', '00038', '00047', '00046', '00021', '00010', '00028', '00020', '00029', '00048', '00007', '00023', '00014', '00034', '00035', '00054', '00027', '00060', '00031', '00061', '00025', '00018', '00039', '00002', '00013', '00003', '00009', '00044', '00050', '00012', '00051', '00022', '00030', '00042', '00006', '00052', '00008', '00045', '00043', '00053', '00017', '00056', '00049', '00040', '00005', '00057', '00001', '00004', '00016', '00036', '00011', '00000', '00041', '00059', '00032', '00015', '00026', '00033', '00024', '00019', '00058', '00037']


In [6]:
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))

Unique Labels: 62
Total Images: 4575
